In [1]:
import GPy
import numpy as np
import pandas as pd

In [25]:
df = pd.read_csv('fulldf.csv',encoding = 'unicode_escape') 

In [26]:
df2 = df[['_chemical_formula_weight','_space_group_IT_number', '_symmetry_cell_setting', '_symmetry_space_group_name_H-M',
         '_cell_angle_alpha', '_cell_angle_beta', '_cell_angle_gamma', '_diffrn_reflns_theta_full',
         '_diffrn_reflns_theta_max', '_diffrn_reflns_theta_min', '_exptl_absorpt_coefficient_mu',
         '_exptl_absorpt_correction_T_max', '_exptl_absorpt_correction_T_min', 'num_atoms']]

In [42]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
space_vals_temp = df2['_symmetry_space_group_name_H-M']
df2['_symmetry_space_group_name_H-M'] = space_vals_temp.astype(str)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(df2['_symmetry_space_group_name_H-M'])
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
space_vals_encoded = onehot_encoder.fit_transform(integer_encoded)
df2['_symmetry_space_group_name_H-M'] = integer_encoded
#print(df2['_symmetry_space_group_name_H-M'])

 /Users/nilamandal/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 /Users/nilamandal/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning:The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
 /Users/nilamandal/opt/anaconda3/lib/python3.7/site-packages/ip

In [47]:
sorted_eigs = df['eig']
conv_eigs = []
for val in sorted_eigs:
    temp = []
    eig_str = val[1:-1] # get rid of brackets
    try:
        temp = [float(x) for x in eig_str.split(',')]
    except:
        t = [complex(''.join(a.split())) for a in eig_str.split(',')]  
        temp = [z.real for z in t]
    conv_eigs.append(temp)
xdf = pd.DataFrame.from_records(conv_eigs)
df2['eig']=conv_eigs
print(df2['eig'])

0        [4267.699976133702, 3770.9038214909883, 542.64...
1        [982.2658584535666, 516.9357657927295, 424.237...
2        [486.7720362544897, 174.73302694103933, 140.11...
3        [918.2482181592767, 530.0995402948571, 461.672...
4        [466.96443464306896, 186.28089562354964, 144.0...
                               ...                        
12495    [477.4976439857017, 332.83733650025306, 155.70...
12496    [353.3725696632276, 188.65121169887047, 112.85...
12497    [446.7017954220414, 337.52969644375935, 240.28...
12498    [529.2210104049125, 363.2248817129924, 164.113...
12499    [427.9109952598827, 154.97176322150952, 128.48...
Name: eig, Length: 12500, dtype: object


In [48]:
tempeig=list(df2['eig'])

for x in range(208):
    e='eig'+str(x)
    df2[e]=[row[x] for row in tempeig]
df2=df2.drop('eig', axis=1) 
print(df2)

       _chemical_formula_weight  _space_group_IT_number  \
0                        454.29                     2.0   
1                        329.97                     2.0   
2                        339.34                     2.0   
3                        241.11                     9.0   
4                        203.24                    14.0   
...                         ...                     ...   
12495                    308.40                     2.0   
12496                    646.76                     2.0   
12497                    458.77                     2.0   
12498                    174.21                    14.0   
12499                    255.30                     2.0   

       _symmetry_cell_setting  _symmetry_space_group_name_H-M  \
0                         3.0                              34   
1                      -100.1                              34   
2                         3.0                              34   
3                         1.0  

In [49]:
y = df['bandgaps'][:5000]
x = df2[:5000]
#x = xdf[:5000]
y = y.to_numpy()
y= y - 3.047
y= np.reshape(y,(5000,1))
x = x.to_numpy()
#print(y)


#X = np.random.uniform(-3.,3.,(20,1))
#Y = np.sin(X) + np.random.randn(20,1)*0.05
#kernel = GPy.kern.Matern32(input_dim=14, variance=1, lengthscale=16)
kernel = GPy.kern.RBF(input_dim=222, variance=1., lengthscale=2.) # using RBF kernel  
#
#

In [50]:
m = GPy.models.GPRegression(x,y,kernel)

In [51]:
from IPython.display import display
display(m)


GP_regression.,value,constraints,priors
rbf.variance,1.0,+ve,
rbf.lengthscale,2.0,+ve,
Gaussian_noise.variance,1.0,+ve,


In [52]:
m.optimize(messages=True)

In [40]:
x_test= df2[11000:]
y_test= df['bandgaps'][11000:]
x_test= x_test.to_numpy()
y_test= y_test.to_numpy()
y_test= y_test - 3.047
xmean, xvar= m.predict(x_test)
print(xmean)
print(xvar)

[[ 0.71244448]
 [ 0.02097407]
 [-0.31767554]
 ...
 [-0.52902311]
 [ 0.20883225]
 [ 0.19872397]]
[[0.83815756]
 [0.83972055]
 [0.71354629]
 ...
 [0.73489081]
 [0.70849782]
 [0.80174997]]


In [41]:
mse = (np.square(y_test - xmean)).mean()
print(mse)
print(np.sqrt(mse))
mae= np.abs(y_test-xmean).mean()
print(mae)

1.2285318981770945
1.1083915816069223
0.8715026930019631
